# ENVIRONMENT SETUP:

In [ ]:
PATH_BASE="/content/drive/MyDrive/Project III"

In [ ]:
%cd /content/drive/MyDrive/Project III
%pwd

/content/drive/MyDrive/Project III


UsageError: Line magic function `%git` not found.


In [ ]:
!git clone https://github.com/snehalnair/als-recommender-pyspark.git

Cloning into 'als-recommender-pyspark'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 4), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [ ]:
%cd als-recommender-pyspark
%ls

[Errno 2] No such file or directory: 'als-recommender-pyspark'
/content/drive/My Drive/Project III/als-recommender-pyspark
movies.csv  ratings.csv  Recommendation_Engine_MovieLens.ipynb  tags.csv


In [50]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 15.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=5664434e4ee682c9944d52d9278b72821ec4102303b91852e06a3a62778b7a16
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


# IMPLEMENTATION:

In [ ]:
import pandas as pd

In [48]:
# import rating data and look at columsn
ratings = pd.read_csv("ratings.csv")
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [56]:
# def get_mat_sparsity(ratings):
#     # Count the total number of ratings in the dataset
#     count_nonzero = ratings.loc[:,"rating"].count()

#     # Count the number of distinct userIds and distinct movieIds
#     total_elements = ratings.loc[:,"userId"].unique().size * ratings.loc[:,"movieId"].unique().size

#     # Divide the numerator by the denominator
#     sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
#     print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")
    
# get_mat_sparsity(ratings)

def get_mat_sparsity(ratings):
    # Count the total number of ratings in the dataset
    count_nonzero = ratings.select("rating").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = ratings.select("userId").distinct().count() * ratings.select("movieId").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")
    
get_mat_sparsity(ratings)

The ratings dataframe is  98.30% sparse.


In [54]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendations').getOrCreate()
movies = spark.read.csv("movies.csv",header=True)
ratings = spark.read.csv("ratings.csv",header=True)
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [52]:
# Join both the data frames to add movie data into ratings
movie_ratings = ratings.join(movies, ['movieId'], 'left')
movie_ratings.show()

+-------+------+------+---------+--------------------+--------------------+
|movieId|userId|rating|timestamp|               title|              genres|
+-------+------+------+---------+--------------------+--------------------+
|      1|     1|   4.0|964982703|    Toy Story (1995)|Adventure|Animati...|
|      3|     1|   4.0|964981247|Grumpier Old Men ...|      Comedy|Romance|
|      6|     1|   4.0|964982224|         Heat (1995)|Action|Crime|Thri...|
|     47|     1|   5.0|964983815|Seven (a.k.a. Se7...|    Mystery|Thriller|
|     50|     1|   5.0|964982931|Usual Suspects, T...|Crime|Mystery|Thr...|
|     70|     1|   3.0|964982400|From Dusk Till Da...|Action|Comedy|Hor...|
|    101|     1|   5.0|964980868|Bottle Rocket (1996)|Adventure|Comedy|...|
|    110|     1|   4.0|964982176|   Braveheart (1995)|    Action|Drama|War|
|    151|     1|   5.0|964984041|      Rob Roy (1995)|Action|Drama|Roma...|
|    157|     1|   5.0|964984100|Canadian Bacon (1...|          Comedy|War|
|    163|   

In [57]:
get_mat_sparsity(ratings)

The ratings dataframe is  98.30% sparse.


In [59]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 2020)
train.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|   1009|   3.0|964981775|
|     1|    101|   5.0|964980868|
|     1|   1023|   5.0|964982681|
|     1|   1024|   5.0|964982876|
|     1|   1025|   5.0|964982791|
|     1|   1029|   5.0|964982855|
|     1|   1030|   3.0|964982903|
|     1|   1032|   5.0|964982791|
|     1|   1042|   4.0|964981179|
|     1|   1049|   5.0|964982400|
|     1|   1073|   5.0|964981680|
|     1|   1080|   5.0|964981327|
|     1|   1089|   5.0|964982951|
|     1|   1092|   5.0|964983484|
|     1|   1097|   5.0|964981680|
|     1|   1136|   5.0|964981327|
|     1|   1196|   5.0|964981827|
|     1|   1197|   5.0|964981872|
|     1|   1198|   5.0|964981827|
+------+-------+------+---------+
only showing top 20 rows



In [62]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ALS model
als = ALS(
         userCol="userId", 
         itemCol="movieId",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)

In [63]:
# Import the requisite packages
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [64]:
# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [65]:
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [66]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [68]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)
#Extract best model from the cv model above
best_model = model.bestModel
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

IllegalArgumentException: ignored

In [67]:
print("**Best Model**")
# Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())
# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

**Best Model**


NameError: ignored